4-19-25

Some data adjustments before performing a lead time analysis. 
#1 Adding a months since baseline column using global average months elapsed between visits.
#2 Combining all progressors into a single csv


In [1]:
import pandas as pd
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from itertools import chain
from xgboost_leadtime import evaluate_lead_time, get_first_progression_visit, create_delta_features_truncated, create_target_variable_ad, create_target_variable_mci, add_time_dimension

In [2]:
# Define the path to the saved models folder
saved_models_path = r"c:\\Users\\Andrew\\Desktop\\AGE\\saved_models"

# Create a dictionary to store the CN to MCI progression models
cn_mci_models = {}

# Loop through the files in the saved models folder
for file_name in os.listdir(saved_models_path):
    if "CN_MCI_model_MCI" in file_name:  # Filter for CN to MCI models
        visit_number = int(file_name.split("visit")[0])  # Extract the visit number
        model_path = os.path.join(saved_models_path, file_name)
        
        # Load the model and add it to the dictionary
        with open(model_path, "rb") as model_file:
            cn_mci_models[visit_number] = pickle.load(model_file)

# Print the dictionary keys to confirm
print(f"Loaded CN to MCI models for visits: {list(cn_mci_models.keys())}")

Loaded CN to MCI models for visits: [2, 3, 4, 5, 6]


In [3]:
df = pd.read_csv('Dataset_1\\mci_progressors.csv')
df = add_time_dimension(df, months_between_visits= 15.7)

In [4]:
lead_times = evaluate_lead_time(df, cn_mci_models, progression_type='MCI')

if len(lead_times) == 0:
    print("No lead times calculated. Check thresholds or data.")
else:
    mean_lead_time = np.mean(lead_times)
    positive_lead = sum(lt > 0 for lt in lead_times) / len(lead_times)
    
    print(f"Mean Lead Time: {mean_lead_time:.1f} months")
    print(f"Percentage Predicted Earlier: {positive_lead * 100:.1f}%")
    
    # Plot
    plt.hist(lead_times, bins=20)
    plt.title("Lead Time Distribution (Months Earlier Than Diagnosis)")
    plt.xlabel("Lead Time (Months)")
    plt.ylabel("Frequency")
    plt.show()

c:\Users\Andrew\Desktop\AGE\xgboost_leadtime.py:84: RuntimeWarning: Mean of empty slice
  lambda x: np.nanmean(x) if isinstance(x, list) else np.nan
c:\Users\Andrew\Desktop\AGE\xgboost_leadtime.py:87: RuntimeWarning: All-NaN axis encountered
  lambda x: np.nanmax(x) if isinstance(x, list) else np.nan


NotFittedError: This SimpleImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [5]:
# Load the dataset
df = pd.read_csv('Dataset_1/mci_progressors.csv')
print("Dataset head:")
print(df.head())

# Check for progression
df['progressed_to_mci'] = df.apply(create_target_variable_mci, axis=1)
print(f"Number of patients who progressed to MCI: {df['progressed_to_mci'].sum()}")

# Evaluate lead time
lead_times = evaluate_lead_time(df, cn_mci_models)

# Handle empty lead_times
if len(lead_times) == 0:
    print("No lead times calculated. Check data or models.")
else:
    mean_lead_time = np.mean(lead_times)
    positive_lead = sum(lt > 0 for lt in lead_times) / len(lead_times)
    print(f"Mean Lead Time: {mean_lead_time:.2f} visits")
    print(f"Percentage Predicted Earlier: {positive_lead * 100:.1f}%")

Dataset head:
           ID  Prog_ID Progression  SEX  EDUC  ALCOHOL           BMI  \
0  NACC257305        1      (0, 1)    2  14.0        0  [21.0, 19.2]   
1  NACC308174        1      (0, 1)    2  15.0        0  [22.3, 24.5]   
2  NACC327105        1      (0, 1)    2  16.0        2  [19.1, 19.6]   
3  NACC422228        1      (0, 1)    1  18.0        0  [28.0, 27.8]   
4  NACC469616        1      (0, 1)    1  13.0        0  [23.7, 24.8]   

           MMSE     GDS         CDR  ... HYPERCHO HYPERTEN B12DEF DEPD ANX  \
0  [30.0, 30.0]  [3, 2]  [0.5, 0.5]  ...        0        0      0    0   0   
1  [29.0, 25.0]  [1, 2]  [0.0, 0.5]  ...        1        1      0    0   0   
2  [30.0, 30.0]  [6, 3]  [0.0, 0.0]  ...        0        0      0    0   0   
3  [29.0, 29.0]  [7, 9]  [0.5, 0.5]  ...        1        0      0    0   0   
4  [29.0, 30.0]  [3, 3]  [0.0, 0.5]  ...        1        1      0    0   0   

  NACCTBI SMOKYRS RACE age HISPANIC  
0       0    40.0  2.0  65        0  
1       

KeyError: 'months_since_baseline'